<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0703125
Current benign train loss: 2.396238327026367

Current batch: 100
Current benign train accuracy: 0.1953125
Current benign train loss: 2.0463979244232178

Current batch: 200
Current benign train accuracy: 0.3359375
Current benign train loss: 1.9019907712936401

Current batch: 300
Current benign train accuracy: 0.2578125
Current benign train loss: 1.8845731019973755

Current batch: 400
Current benign train accuracy: 0.3359375
Current benign train loss: 1.771867275238037

Current batch: 500
Current benign train accuracy: 0.3125
Current benign train loss: 1.8545849323272705

Current batch: 600
Current benign train accuracy: 0.4140625
Current benign train loss: 1.644195556640625

Current batch: 700
Current benign train accuracy: 0.3359375
Current benign train loss: 1.6762957572937012

Current batch: 800
Current benign train accuracy: 0.375
Current benign train loss: 1.6101844310760498

Current batch: 900
Current b


Current batch: 800
Current benign train accuracy: 0.796875
Current benign train loss: 0.48801732063293457

Current batch: 900
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2931864261627197

Current batch: 1000
Current benign train accuracy: 0.8515625
Current benign train loss: 0.39712250232696533

Current batch: 1100
Current benign train accuracy: 0.8671875
Current benign train loss: 0.366870641708374

Total benign train accuarcy: 84.78333333333333
Total benign train loss: 511.5638933032751

[ Test epoch: 5 ]

Test accuarcy: 81.59
Test average loss: 0.0055214804500341415
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.8671875
Current benign train loss: 0.39021286368370056

Current batch: 100
Current benign train accuracy: 0.8671875
Current benign train loss: 0.38643723726272583

Current batch: 200
Current benign train accuracy: 0.875
Current benign train loss: 0.398639440536499

Current batch: 300
Current benign train accurac


Current batch: 200
Current benign train accuracy: 0.9296875
Current benign train loss: 0.21404829621315002

Current batch: 300
Current benign train accuracy: 0.8671875
Current benign train loss: 0.39237746596336365

Current batch: 400
Current benign train accuracy: 0.890625
Current benign train loss: 0.32149970531463623

Current batch: 500
Current benign train accuracy: 0.875
Current benign train loss: 0.3405160903930664

Current batch: 600
Current benign train accuracy: 0.921875
Current benign train loss: 0.30886924266815186

Current batch: 700
Current benign train accuracy: 0.890625
Current benign train loss: 0.3131578266620636

Current batch: 800
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3268905282020569

Current batch: 900
Current benign train accuracy: 0.875
Current benign train loss: 0.29485929012298584

Current batch: 1000
Current benign train accuracy: 0.8828125
Current benign train loss: 0.25192904472351074

Current batch: 1100
Current benign train


Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2770628333091736

Current batch: 1100
Current benign train accuracy: 0.953125
Current benign train loss: 0.16359078884124756

Total benign train accuarcy: 91.05
Total benign train loss: 305.7368098720908

[ Test epoch: 16 ]

Test accuarcy: 84.72
Test average loss: 0.005095560228824615
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.20229481160640717

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.2885083556175232

Current batch: 200
Current benign train accuracy: 0.9296875
Current benign train loss: 0.18068425357341766

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.25644347071647644

Current batch: 400
Current benign train accuracy: 0.8984375
Current benign train loss: 0.26342839002609253

Current batch: 500
Current benign train accuracy: 0.890625


Current batch: 400
Current benign train accuracy: 0.921875
Current benign train loss: 0.22278732061386108

Current batch: 500
Current benign train accuracy: 0.8828125
Current benign train loss: 0.34848788380622864

Current batch: 600
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2358437329530716

Current batch: 700
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2717060148715973

Current batch: 800
Current benign train accuracy: 0.90625
Current benign train loss: 0.2838907837867737

Current batch: 900
Current benign train accuracy: 0.953125
Current benign train loss: 0.17544448375701904

Current batch: 1000
Current benign train accuracy: 0.9140625
Current benign train loss: 0.27522510290145874

Current batch: 1100
Current benign train accuracy: 0.90625
Current benign train loss: 0.2883571684360504

Total benign train accuarcy: 91.626
Total benign train loss: 288.11255428195

[ Test epoch: 22 ]

Test accuarcy: 85.38
Test average loss: 0.004

...


Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.01051038596779108

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.013892273418605328

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.003366479417309165

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.009564829058945179

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.005722559988498688

Current batch: 1100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.014682643115520477

Total benign train accuarcy: 99.57266666666666
Total benign train loss: 16.736397790082265

[ Test epoch: 72 ]

Test accuarcy: 92.56
Test average loss: 0.0028769317872822287
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.02874183841049671

Current batch: 100
Current benign train accuracy: 0.9921875



Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.010149992071092129

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.0185281652957201

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.010119454935193062

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.013795276172459126

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.010889205150306225

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.026723934337496758

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.009203555062413216

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.021450242027640343

Current batch: 800
Current benign train accuracy: 0.984375
Current benign train loss: 0.039427731186151505

Current batch: 900
Current benign train accuracy: 1.0


Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.0012436691904440522

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0021436940878629684

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.0014473474584519863

Total benign train accuarcy: 99.85133333333333
Total benign train loss: 7.562653267377755

[ Test epoch: 83 ]

Test accuarcy: 93.31
Test average loss: 0.0025457851222716273
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.005385215859860182

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0005629428196698427

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0018300832016393542

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0025655406061559916

Current batch: 400
Current benign train accuracy: 1.0
Current be


Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0014632845995947719

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.002999114105477929

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0024365736171603203

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0018890508217737079

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0021555323619395494

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.006432527210563421

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0011856493074446917

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.00047962545068003237

Total benign train accuarcy: 99.89533333333334
Total benign train loss: 5.341097331838682

[ Test epoch: 89 ]

Test accuarcy: 93.43
Test average loss: 0.0025070738811


Test accuarcy: 93.38
Test average loss: 0.0024823410419747235
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02382364124059677

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.001722151879221201

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0008011515601538122

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.005145088769495487

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0008050461765378714

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.005698574706912041

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0015877678524702787

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.005153147503733635

Current batch: 800
Current benign train accuracy: 1.0
Current benign tra